# Sub-word-level Tokenizing

Recall flaws of 
1. Word-based approaches

* Very large vocabularies
* Large quantity of out-of-vocabulary tokens
* Loss of meaning across very similar words (dog, dogs, sun, sunny, hap, happy)

2. Character-based approaches

* Very long sequences
* Less meaningful individual tokens

So,

A middle-ground between the two is found.

1. Frequently used words should not be split (into subwords)
2. Rare words should be split into "meaningful" subwords

`dog` -> `dog`
`dogs` -> `dog` `s`

see [hf video on it](https://www.youtube.com/watch?v=zHvTiHr506c)

Good example given is `tokenization`: 

`token` -- `ization`  

`token`: Token, tokens, tokenizing, tokenization, tokenized, tokenizes, tokenizable, tokenizability  
`ization`: tokenization, modernization, globalization, industrialization, organization, realization,   utilization, ...  

* Sub-word-based tokenizers typically identify when a word is split, by special character padding: 

`token` `##ization` (BERT)

The number of `#` may or may not refer to the count of preceding letters it was split from (in this case it doesn't)

# Sub-Word Algorithms

1. WordPiece (BERT, DistilBERT,  ...)
2. Unigram (XLNET, ALBERT)
3. Byte-Pair Encoding (RoBERTa, GPT-2+, T5, ...)
3. [SentencePiece](https://huggingface.co/docs/transformers/tokenizer_summary#sentencepiece+) (sorta T5, ...) ((all that use it also use Unigram))

## WordPiece

## Unigram Tokenization

Overall strategy is to start with a very large vocabulary, and iteratively shrink it. Each iteration a unigram loss is calculated and the bottom _p_ tokens are reduced.

* Unigram model is stats model: $$P(t1, t2, t3, ... tN)$$

* Unigram model assumes that the occurence of each word is independent of its previous word. Thus, can calulate overall probability of next word: 

$$P(t1, t2, t3, ... tN)= P(t1) x P(t2) x P(t3) x ... x P(TN)$$

i.e. the probability of a text is the probability of the tokens that compose it. This means that it can't perform meaningful text generation: it will always predict the single highest probability token. So, what's it good for? 

It is a useful model to estimate the relative likelihood of different phrases. (..?)  

**TODO Fill iterations**

* each iteration, calculate probabilities of each token 
* remove one that impacts loss the least && is not an elementary token (as this would prevent ever spelling a word that contains it)


## Byte-Pair Encoding (BPE) Tokenizaiton

Idea is originally text compression algorithm. 

Psuedo-algorithm: 

* Take a corpus, split into all words, then into all characters
* count each pair frequency, finding most common pair(s)
* take most common pair as a token and add to vocabulary
* iterate until desired vocab size is reached 

* [HuggingFace Video](https://www.youtube.com/watch?v=HEikzVL-lZU)

## SentencePiece

# Imports

In [1]:
%load_ext kedro.ipython
%reload_kedro

from typing import Any, Dict, List, Tuple

import re


from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoTokenizer

[02/12/24 01:07:30] INFO     Resolved project path as: c:\Users\bcech\Documents\ML\tolkenizers.     ]8;id=305817;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=79656;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py#149\149]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/12/24 01:07:31] INFO     Kedro project tolkenizers                                              ]8;id=891713;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=241776;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py#119\119]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=946696;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=371796;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py#120\120]8;;\
                             'pipelines'                                                                           

[02/12/24 01:07:33] INFO     Registered line magic 'run_viz'                                        ]8;id=946367;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=548497;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py#126\126]8;;\

                    INFO     Resolved project path as: C:\Users\bcech\Documents\ML\tolkenizers.     ]8;id=93606;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=782038;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py#149\149]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Kedro project tolkenizers                                              ]8;id=744358;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=710089;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py#119\119]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=598980;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=446929;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py#120\120]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=285075;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=724180;file://c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\kedro\ipython\__init__.py#126\126]8;;\

[02/12/24 01:07:35] WARNING  c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\tqdm\auto.py:21:   ]8;id=3992;file://c:\Users\bcech\anaconda3\envs\tok\Lib\warnings.py\warnings.py]8;;\:]8;id=648350;file://c:\Users\bcech\anaconda3\envs\tok\Lib\warnings.py#109\109]8;;\
                             TqdmWarning: IProgress not found. Please update jupyter and                           
                             ipywidgets. See                                                                       
                             https://ipywidgets.readthedocs.io/en/stable/user_install.html                         
                               from .autonotebook import tqdm as notebook_tqdm                                     
                                                                                                                   

## SentencePiece

In [2]:
model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)

In [3]:
s = "Stars, hide your fires; Let not light see my black and deep desires."

In [4]:
encoded = tokenizer(s)
print(f"{encoded = }")
encoded = tokenizer.encode(s)
print(f"{encoded = }")
encoded_plus = tokenizer.encode_plus(s)
print(f"{encoded_plus = }")

encoded = {'input_ids': [101, 3340, 1010, 5342, 2115, 8769, 1025, 2292, 2025, 2422, 2156, 2026, 2304, 1998, 2784, 14714, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
encoded = [101, 3340, 1010, 5342, 2115, 8769, 1025, 2292, 2025, 2422, 2156, 2026, 2304, 1998, 2784, 14714, 1012, 102]
encoded_plus = {'input_ids': [101, 3340, 1010, 5342, 2115, 8769, 1025, 2292, 2025, 2422, 2156, 2026, 2304, 1998, 2784, 14714, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


and to grab the actual tokens

In [6]:
print(tokenizer.convert_ids_to_tokens(encoded))

['[CLS]', 'stars', ',', 'hide', 'your', 'fires', ';', 'let', 'not', 'light', 'see', 'my', 'black', 'and', 'deep', 'desires', '.', '[SEP]']


What about other special characters and padding? 

In [15]:
s = ["In the caldron boil and bake;",
    "Eye of newt and toe of frog",
    "Wool of bat and tongue of dog",
    "Adder's fork and blind-worm's sting",
    "Lizard's leg and howlet's wing",
    "For a charm of powerful trouble",
    "Like a hell-broth boil and bubble."
]
encoded = tokenizer(s, padding=True, add_special_tokens=True)

# print ids
for a in encoded['input_ids']:
    print(a)

# print attention mask
for a in encoded['attention_mask']:
    print(a)

for a in encoded['input_ids']:
    print(tokenizer.convert_ids_to_tokens(a))

[101, 1999, 1996, 10250, 19440, 26077, 1998, 8670, 3489, 1025, 102, 0, 0, 0]
[101, 3239, 1997, 25597, 1998, 11756, 1997, 10729, 102, 0, 0, 0, 0, 0]
[101, 12121, 1997, 7151, 1998, 4416, 1997, 3899, 102, 0, 0, 0, 0, 0]
[101, 5587, 2121, 1005, 1055, 9292, 1998, 6397, 1011, 15485, 1005, 1055, 12072, 102]
[101, 15450, 1005, 1055, 4190, 1998, 22912, 3388, 1005, 1055, 3358, 102, 0, 0]
[101, 2005, 1037, 11084, 1997, 3928, 4390, 102, 0, 0, 0, 0, 0, 0]
[101, 2066, 1037, 3109, 1011, 22953, 2705, 26077, 1998, 11957, 1012, 102, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
['[CLS]', 'in', 'the', 'cal', '##dron', 'boil', 'and', 'ba', '##ke', ';', '[SEP]', '[PAD]', '[PAD]', '[PAD]']
['[CLS]', 'eye', 'of', 'newt', 'and', 'toe', 'of', 'frog', '[S